In [49]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
import csv
import pandas as pd

chat = ChatOpenAI(model="gpt-3.5-turbo-1106",
                  temperature=1,
                  
                  max_tokens=4095,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0)

def gen_data(raw_content):
    """
    使用LangChain GPT-3.5调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-3.5模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国铁路专家，尤其擅长《铁路旅客运输规程》内容。

        接下来，你收到的都是关于《铁路旅客运输规程》的内容，你需要整理成问答格式，并生成大模型训练的内容和格式。
        
        示例输入：
        
        第四条 车票是铁路旅客运输合同的凭证，可以采取纸质形式或者电子数据形式，一般应当载明发站、到站、车次、车厢号、席别、席位号、票价、开车时间等主要信息。铁路运输企业与旅客另有约定的，按照其约定。
        
        期待结果：
        
        content:"什么是车票？"
        summary:"车票是铁路旅客运输合同的凭证，可以采取纸质形式或者电子数据形式，一般应当载明发站、到站、车次、车厢号、席别、席位号、票价、开车时间等主要信息。铁路运输企业与旅客另有约定的，按照其约定。"
        
        content:"车票包括哪些信息？"
        summary:"车票包括发站、到站、车次、车厢号、席别、席位号、票价、开车时间等主要信息"
        
        返回格式要求：
        content:"{问题}"
        summary:"{答案}"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

# 初始化一个空列表用于存储原始内容数据
raw_content_data = []

# 读取文件并分割数据样例
with open('./data/lvke.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    # 使用连续的换行符('\n\n')作为分隔符来分割文本
    data_samples = content.split('\n\n')

    # 遍历分割后的数据样例并添加到列表中
    for sample in data_samples:
        # 移除每个样例中的额外空白字符（如果有的话）
        cleaned_sample = sample.strip()
        # 仅添加非空样例
        if cleaned_sample:
            raw_content_data.append(cleaned_sample)
            
def dataset_parser(ai_message_content):
    """
    解析由gen_data函数生成的ai_message.content,提取content和summary。

    :param ai_message_content: gen_data函数返回的文本。
    :return: 提取的content和summary。
    """
    # 使用换行符分割文本以区分content和summary
    parts = ai_message_content.split('\n')

    # 提取content和summary
    content = parts[0].split(':', 1)[1].strip('"')
    summary = parts[1].split(':', 1)[1].strip('"')
    

    return content, summary

def generate_question_summary_pairs(content, summary):
    system_content1 ="""
        实际用户提问的时候，不会直接说“铁路运输企业应当如何为旅客提供服务？”，因此，请给出20种可能提问的方式,如
        "铁路运输企业如何服务旅客？",
        "旅客在铁路运输中应享受哪些服务？",
        "铁路公司提供什么服务来确保旅客满意？",
        "如何评价铁路运输企业对旅客的服务？",
        "铁路运输中，旅客期待哪些服务？",
        "铁路服务中包含哪些旅客服务项目？",
        "怎样的铁路服务能满足旅客需求？",
        "铁路运输企业怎样提升旅客服务质量？",
        "旅客在铁路旅行中应享受何种服务体验？",
        "铁路运输对旅客有哪些服务承诺？",
        "如何提高铁路运输旅客的服务满意度？",
        "铁路公司如何确保旅客服务的高标准？",
        "在铁路运输中，如何向旅客提供优质服务？",
        "铁路旅行中旅客服务包括哪些内容？",
        "铁路运输企业为旅客提供哪些便利和服务？",
        "铁路旅客服务的主要内容有哪些？",
        "铁路运输中旅客最期待哪些服务？",
        "如何在铁路运输中改善旅客服务？",
        "铁路运输企业在服务旅客方面有哪些规定？",
        "铁路运输中，为旅客提供哪些基本服务？"
        写入questions列表
        """

    messages1 = [
        SystemMessage(
            content=system_content1
        ),
        HumanMessage(
            content=content
        )
    ]
    ai_message1 = chat(messages1)

    # Split the string into individual questions and create a list 
    questions_list = ai_message1.content.split('\n')
    
    # Remove any empty strings from the list and remove the numbers at the beginning  
    questions_list = [question.split(' ', 1)[1].strip() for question in questions_list if question.strip()]
    #df = pd.DataFrame({
     #   'Question': questions_list,
     #   'Summary': [summary] * len(questions_list)
     #   })
    # 创建提问和总结的配对
    question_summary_pairs = [(question, summary) for question in questions_list]
    
    return question_summary_pairs

import csv
import datetime
import os

def main():
    # 确保 data 目录存在
    if not os.path.exists('./data'):
        os.makedirs('./data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('./data/lvke.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)

    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"./data/lvkeyunshuguicheng_dataset_{timestamp}.csv"

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in raw_content_data:
            #ai_message_content = []
            # 调用 gen_data 方法得到 ai_message_content
            ai_message_content = gen_data(raw_content)

            # 解析 ai_message_content 得到 content 和 summary
            content, summary = dataset_parser(ai_message_content)
            
            print("Content:", content)
            print("Summary:", summary)

            # 调用 generate_question_summary_pairs 得到20组 pairs
            pairs = generate_question_summary_pairs(content, summary)

            # 将 pairs 写入 csv 文件
            for pair in pairs:
                writer.writerow(pair)
                
main()

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.


Content: 旅客托运行李应当遵守哪些规定？
Summary: 旅客托运行李应当遵守国家禁止运输或者限制运输的相关规定，不得夹带货币、珍贵文物、金银珠宝、档案材料、证书证件等贵重物品。
Content:  "行李应当如何运送？
Summary:  "行李应当随旅客所乘列车或者就近列车运送，如遇特殊情况的，铁路运输企业应当向旅客作出说明。
Content: 铁路运输企业应当为旅客办理行李托运变更手续提供怎样的条件？
Summary: 铁路运输企业应当为旅客办理行李托运变更手续提供便利条件。
Content: 铁路运输企业应当如何处理旅客的行李？
Summary: 铁路运输企业应当及时通知旅客领取到达的行李。行李从运到日起，铁路运输企业应当至少免费保管3日；逾期到达的行李应当至少免费保管10日。因行李损失或者不可抗力等原因应当适当增加免费保管日数。对时限的约定也可依照双方约定执行。
Content: 行李运输过程中，如果行李发生损坏、丢失或者延误，旅客有什么要求？
Summary: 旅客要求出具行李运输损失凭证，铁路运输企业应当及时提供。
Content: 铁路运输企业应当如何处理自然灾害、恶劣天气、设备设施故障以及安全事故对旅客出行产生重大影响的情形？
Summary: 铁路运输企业应当针对这些情况依法制定相应的应急预案，配备必要的应急物品，并定期组织演练。
Content: 列车不能继续运行时，铁路运输企业应当如何处理受阻旅客？
Summary: 铁路运输企业应当妥善安排受阻旅客，并及时公告铁路旅客运输相关业务停办、恢复等信息。因线路中断影响旅行，旅客要求出具证明的，铁路运输企业应当及时提供。
Content: 铁路运输企业线路中断后该如何处理旅客行李？
Summary: 铁路运输企业应当妥善保管承运的行李，并及时通知旅客，与旅客协商处理方案。
Content: 铁路运输企业在旅客急病、分娩、遇险时应该采取怎样的措施？
Summary: 铁路运输企业应当积极采取救助措施并做好记录。
Content: 铁路运输企业对旅客在运输过程中的人身伤亡和财产损失承担怎样的责任？
Summary: 铁路运输企业对运输过程中旅客的人身伤亡和财产损失，依法承担民事责任。
Content: 旅客对铁路旅客运输服务质量问题有哪些投诉权利？
Summary: 旅客有权就铁路旅客运输服务